In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle (1).json" with length 69 bytes


In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

In [8]:
!if [ ! -d "/content/automathon-deepfake" ]; then kaggle competitions download -c automathon-deepfake -p /content/automathon-deepfake; unzip /content/automathon-deepfake/automathon-deepfake.zip -d /content/automathon-deepfake; else echo "Directory /content/automathon-deepfake already contains data."; fi



Directory /content/automathon-deepfake already contains data.


In [9]:
!pip install torch torchvision


In [10]:
!mv /content/automathon-deepfake/dataset/experimental_dataset/metadata.json /content/automathon-deepfake/dataset

mv: cannot stat '/content/automathon-deepfake/dataset/experimental_dataset/metadata.json': No such file or directory


In [11]:
%cd /content/automathon-deepfake
%mkdir frames

/content/automathon-deepfake


In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import Compose, ToPILImage, Resize, CenterCrop, ToTensor, Normalize
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_tensor, normalize
from torchvision.io import read_video
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
import glob
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
from torchvision.utils import save_image
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import torchvision.transforms.functional as F
from torchsummary import summary
import torch.optim as optim

In [2]:
DATASET_VIDEO_PATH = "/content/automathon-deepfake/dataset/experimental_dataset"
DATASET_METADATA_PATH = "/content/automathon-deepfake/dataset/metadata.json"
FRAME_SAVE_PATH = "/content/automathon-deepfake/frames"
FRAME_RATE = 1  # Frame rate to sample (e.g., 1 frame per second)

# Make sure the frame save directory exists
os.makedirs(FRAME_SAVE_PATH, exist_ok=True)

# Load video metadata
df_labels = pd.read_json(DATASET_METADATA_PATH, orient='index')
df_labels.reset_index(inplace=True)
df_labels.columns = ['Filename', 'Label']
df_labels['label_value'] = np.where(df_labels['Label'] == 'real', 1, 0)

In [3]:
class VideoDataset(Dataset):
    def __init__(self, dataframe, root_dir, sequence_length=10, transform=None):
        """
        Args:
            dataframe (DataFrame): DataFrame containing video filenames and labels.
            root_dir (str): Directory path where video files are stored.
            sequence_length (int): Number of frames to extract from each video.
            transform (callable, optional): Optional transform to be applied on a frame.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.sequence_length = sequence_length
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        video_filename = self.dataframe.iloc[idx]['Filename']
        video_path = os.path.join(self.root_dir, video_filename)
        label = self.dataframe.iloc[idx]['label_value']
        # Read video and extract frames
        frames, _, _ = read_video(video_path, pts_unit='sec', start_pts=0, end_pts=10, output_format='TCHW')
        total_frames = len(frames)
        frame_indices = torch.linspace(0, total_frames - 1, steps=self.sequence_length).long()
        selected_frames = frames[frame_indices]

        processed_frames = []
        for frame in selected_frames:
            if self.transform:
                frame = self.transform(frame)
            processed_frames.append(frame)

        frames_tensor = torch.stack(processed_frames)
        return frames_tensor, label

# Example of setting up the dataset and dataloader with transformations
transform = transforms.Compose([
    transforms.ToPILImage(),  # Necessary to convert raw video frame to PIL Image for some transformations
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),  # Convert the PIL Image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

dataset = VideoDataset(df_labels, DATASET_VIDEO_PATH, transform=transform)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [4]:

# Assuming 'df_labels' is your DataFrame containing video filenames and their labels
# Shuffle the DataFrame
df_labels = df_labels.sample(frac=1).reset_index(drop=True)

# Define split sizes
train_size = int(0.7 * len(df_labels))
val_size = int(0.15 * len(df_labels))
test_size = len(df_labels) - train_size - val_size

# Split the DataFrame into train, validation, and test sets
train_df = df_labels[:train_size]
val_df = df_labels[train_size:train_size + val_size]
test_df = df_labels[train_size + val_size:]

# Define a transform pipeline
transform = Compose([
    ToPILImage(),  # Convert raw video frame to PIL Image for transformations
    Resize((256, 256)),
    CenterCrop((224, 224)),
    ToTensor(),  # Convert the PIL Image to a tensor
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

# Create datasets for each set
train_dataset = VideoDataset(train_df, DATASET_VIDEO_PATH, transform=transform)
val_dataset = VideoDataset(val_df, DATASET_VIDEO_PATH, transform=transform)
test_dataset = VideoDataset(test_df, DATASET_VIDEO_PATH, transform=transform)

# Create data loaders for each set
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Now you can use these loaders in your training loop


In [5]:
import torch
from torch.utils.data import DataLoader

# Assuming train_loader is already defined
data_loader = train_loader  # You can replace this with val_loader or test_loader as needed

# Fetch and print the shape of one batch of data and labels from the specified DataLoader
for data, labels in data_loader:
    print(f"Data shape: {data.shape}")  # Shape of the video frames tensor
    print(f"Labels shape: {labels.shape}")  # Shape of the labels tensor
    break  # Only look at the first batch


Data shape: torch.Size([1, 10, 3, 224, 224])
Labels shape: torch.Size([1])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VideoCNN3D(nn.Module):
    def __init__(self):
        super(VideoCNN3D, self).__init__()
        # 3D convolutional layer with input channels = 3 (RGB)
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), padding=1)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Second 3D convolutional layer
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Third 3D convolutional layer
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=1)
        self.pool3 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Flattening and dense layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 10 * 28 * 28, 512)  # Adjust size according to the output of the last pool layer
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 2)  # Assuming binary classification

    def forward(self, x):
        # Invert sequence length and channels dimensions
        x = input_tensor.permute(0, 2, 1, 3, 4)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return torch.argmax(x, dim=1)

# Example initialization and forward pass simulation
model = VideoCNN3D()
print(model)

# Simulate a forward pass
input_tensor = torch.randn(1, 10 ,3, 224, 224)  # Batch size, Channels, Sequence length, Height, Width
output = model(input_tensor)
print("Output shape:", output.shape)


VideoCNN3D(
  (conv1): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1003520, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)
Output shape: torch.Size([1])


In [3]:
"""import torch
import torch.nn as nn
import torch.nn.functional as F

class VideoCNN3D(nn.Module):
    def __init__(self):
        super(VideoCNN3D, self).__init__()
        # First 3D convolutional layer with input channels = 3 (RGB)
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), padding=1)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Second 3D convolutional layer
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Flattening and dense layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 5 * 28 * 28, 512)  # Adjust size according to the output of the last pool layer
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 2)  # Assuming binary classification

    def forward(self, x):
        # Invert sequence length and channels dimensions
        x = x.permute(0, 2, 1, 3, 4)

        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return torch.argmax(x, dim=1)

# Example initialization and forward pass simulation
model = VideoCNN3D()
print(model)

# Simulate a forward pass
input_tensor = torch.randn(1, 10, 3, 224, 224)  # Batch size, Sequence length, Channels, Height, Width
output = model(input_tensor)
print("Output shape:", output.shape)
"""

VideoCNN3D(
  (conv1): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=250880, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2007040 and 250880x512)

In [7]:
model = VideoCNN3D()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [8]:
! pip install av

In [ ]:
import copy
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train()  # Set model to training mode

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs
            labels = labels

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Training Loss: {epoch_loss:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        for inputs, labels in val_loader:
            inputs = inputs
            labels = labels

            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}')

        # Deep copy the model if it has the best validation loss so far
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), 'best_model.pth')
            print('Best model saved.')

        # Step the scheduler
        scheduler.step(val_loss)

    print('Training complete. Best val Loss: {:4f}'.format(best_loss))
    model.load_state_dict(best_model_wts)
    return model

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler)


Epoch 1/25
----------


In [ ]:
import os
import json
from moviepy.editor import VideoFileClip

# Load metadata from a JSON file
with open("/content/automathon-deepfake/dataset/metadata.json", 'r') as file:
    metadata = json.load(file)

def extract_frames(video_path, output_folder, video_name, label, every_n_seconds=1):
    # Create the output folder if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    clip = VideoFileClip(video_path)
    frame_count = int(clip.fps * clip.duration)  # Total number of frames
    for i in range(0, frame_count, int(clip.fps * every_n_seconds)):
        frame = clip.get_frame(i / clip.fps)
        # Frame filename includes video name, label, and frame index
        frame_path = os.path.join(output_folder, f"{video_name}_label_{label}_frame_{i}.png")
        clip.save_frame(frame_path, t=i / clip.fps)

# Define base paths
base_output_folder = "/content/automathon-deepfake"
video_folder = "/content/automathon-deepfake/dataset/experimental_dataset"
frames_folder = os.path.join(base_output_folder, "frames")

# Loop through all video files in the video folder
for video_file in os.listdir(video_folder):
    if video_file.endswith('.mp4'):
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        label = metadata.get(video_file, 'unknown')  # Default to 'unknown' if not found in metadata
        video_output_folder = os.path.join(frames_folder, video_name)  # Output folder for each video

        # Extract frames to the specific video folder
        extract_frames(video_path, video_output_folder, video_name, label)


In [ ]:
!pip install torch torchvision numpy matplotlib pandas

In [ ]:
# Load the model
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
import os
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
import torch
import time

def replace_frames_with_faces(image_path, model, device):
    # Load the image
    image = read_image(image_path).float() / 255  # Normalize the image
    image = image.to(device)

    # Perform inference
    with torch.no_grad():
        prediction = model([image])[0]

    # Filter results with high confidence and person labels
    boxes = prediction['boxes'][prediction['scores'] > 0.8]
    labels = prediction['labels'][prediction['scores'] > 0.8]
    person_boxes = boxes[labels == 1]

    if person_boxes.shape[0] > 0:
        # Use the first detected face
        xmin, ymin, xmax, ymax = person_boxes[0]
        height = ymax - ymin
        face_image = image[:, int(ymin):int(ymin + height), int(xmin):int(xmax)]

        # Save the face image back to the original path
        face_pil = to_pil_image(face_image.cpu())
        face_pil.save(image_path)
        print(f"Processed and saved face image for {image_path}")
    else:
        print(f"No faces detected in {image_path}")

def process_and_replace_faces(base_folder, model, device):
    for subdir in os.listdir(base_folder):
        subdir_path = os.path.join(base_folder, subdir)
        if os.path.isdir(subdir_path):  # Check if it is a directory
            print(f"Processing directory: {subdir_path}")
            for image_file in os.listdir(subdir_path):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(subdir_path, image_file)
                    replace_frames_with_faces(image_path, model, device)
                else:
                    print(f"Skipped non-image file: {image_file}")
        else:
            print(f"Skipped non-directory file: {subdir}")
    print("Processing complete.")

# Assuming the model and device setup from previous example
model.eval()  # Make sure model is in evaluation mode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define your frames directory
frames_directory = '/content/automathon-deepfake/frames'
process_and_replace_faces(frames_directory,model, device)


In [ ]:
def replace_frames_with_faces(image_path, model, device):
    # Load the image
    image = read_image(image_path).float() / 255  # Normalize the image
    image = image.to(device)

    # Perform inference
    with torch.no_grad():
        prediction = model([image])[0]

    # Filter results with high confidence and person labels
    boxes = prediction['boxes'][prediction['scores'] > 0.8]
    labels = prediction['labels'][prediction['scores'] > 0.8]
    person_boxes = boxes[labels == 1]

    if person_boxes.shape[0] > 0:
        # Use the first detected face
        xmin, ymin, xmax, ymax = person_boxes[0]
        height = ymax - ymin
        face_image = image[:, int(ymin):int(ymin + height), int(xmin):int(xmax)]

        # Save the face image back to the original path
        face_pil = to_pil_image(face_image.cpu())
        face_pil.save(image_path)
        print(f"Processed and saved face image for {image_path}")
    else:
        print(f"No faces detected in {image_path}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os
import json
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim


In [ ]:
class DeepfakeDataset(Dataset):
    def __init__(self, img_dir, metadata, transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.
            metadata (dict): Dictionary with labels for the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.metadata = metadata

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = list(self.metadata.keys())[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = read_image(img_path)
        label = self.metadata[img_name]
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Load metadata
with open("/content/automathon-deepfake/dataset/metadata.json", 'r') as file:
    metadata = json.load(file)

# Convert labels to integers if they're not already
for key in metadata:
    metadata[key] = 1 if metadata[key] == 'real' else 0

img_dir = "/content/automathon-deepfake/dataset/frames"
dataset = DeepfakeDataset(img_dir=img_dir, metadata=metadata, transform=transform)


In [ ]:
from torch.utils.data import random_split

# Assuming dataset is already created
total_size = len(dataset)
train_size = int(0.7 * total_size)
valid_size = int(0.2 * total_size)
test_size = total_size - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)


In [ ]:
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Assuming binary classification
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
import matplotlib.pyplot as plt
import torch

train_losses, valid_losses = [], []
best_valid_loss = float('inf')  # Initialize with a very large value
patience = 3  # Number of epochs to wait for improvement
counter = 0  # Counter for patience

num_epochs = 30  # Increase epochs
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for data in train_loader:
        inputs, labels = data['image'], data['label']
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data['image'], data['label']
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()

    # Calculate average losses
    train_loss = train_loss / len(train_loader)
    valid_loss = valid_loss / len(valid_loader)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'Epoch {epoch+1}: Training Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

    # Check for improvement in validation loss
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        counter = 0  # Reset counter
        # Save the model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f'Validation loss has not improved for {patience} epochs. Early stopping...')
            break

# Plot the losses
plt.plot(train_losses, label='Training Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
!pip install torch torchvision tqdm pillow

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from torchvision.transforms import functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [ ]:
def extract_faces_from_video(video_path, output_folder):
    from moviepy.editor import VideoFileClip

    cap = VideoFileClip(video_path)
    frames = []

    with tqdm(total=cap.reader.nframes) as pbar:
        for frame in cap.iter_frames():
            frame = Image.fromarray(frame)
            frame_tensor = F.to_tensor(frame).to(device)
            outputs = mtcnn([frame_tensor])[0]

            for box, score in zip(outputs['boxes'], outputs['scores']):
                if score > 0.9:
                    x1, y1, x2, y2 = [int(val) for val in box]
                    face = frame.crop((x1, y1, x2, y2))
                    frames.append(face)

            pbar.update(1)

    cap.close()

    for i, face in enumerate(frames):
        face.save(f"{output_folder}/face_{i}.jpg")